# Dependencies and imports

In [4]:
import os
import re
import json
from collections import Counter

from openai import OpenAI

import numpy as np
import pandas as pd

from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer

from sentence_transformers import SentenceTransformer

In [5]:
OPENAI_CLIENT = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
PROMPT = 'creare una breve espressione (massimo 5 parole) che riassuma le seguenti frasi'

Mounted at /content/drive


# Utils

In [ ]:
sentence_transformer_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.13k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/402 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def do_clustering(docs):
  docs_embeddings = sentence_transformer_model.encode(docs)
  docs_embeddings = docs_embeddings / np.linalg.norm(docs_embeddings, axis=1, keepdims=True)

  # Perform agglomerative clustering
  clustering_model = AgglomerativeClustering(
      n_clusters=None, metric='cosine', linkage='average', distance_threshold=0.45
  )
  clustering_model.fit(docs_embeddings)
  cluster_assignment = clustering_model.labels_

  clustered_docs = {}
  for doc_index, cluster_id in enumerate(cluster_assignment):
      if cluster_id not in clustered_docs:
          clustered_docs[cluster_id] = []
      clustered_docs[cluster_id].append(docs[doc_index])

  return clustered_docs


def do_topic_extractions(docs, max_topics):
    tfidf = TfidfVectorizer()
    data = tfidf.fit_transform(docs)
    terms = tfidf.get_feature_names_out()

    model = LatentDirichletAllocation(n_components=1)
    model.fit_transform(data)

    topics = []
    for index, component in enumerate(model.components_):

        zipped = zip(terms, component)
        top_nouns_key = sorted(zipped, key=lambda t: t[1], reverse=True)[:max_topics]

        top_nouns_list = list(dict(top_nouns_key).keys())
        topics = topics + top_nouns_list

    return topics

def gen_title_from_docs_list(docs):
  try:
    response = OPENAI_CLIENT.chat.completions.create(
      model="gpt-3.5-turbo-0125",
      messages=[
        {
          "role": "system",
          "content": PROMPT
        },
        {
          "role": "user",
          "content": '\n'.join(docs)
        }
      ],
      max_tokens=256,
    )
    text = response.choices[0].message.content
    return str(text).lower()
  except Exception as e:
    print(e)
    return ''

# Load Data

In [6]:
df1 = pd.read_csv(f"./corpora/reviewer1.csv")
df2 = pd.read_csv(f"./corpora/reviewer2.csv")

# Issues

## Merge lists

In [7]:
r1_list = [i.strip() for i in df1[f'original_text_isssues_detected'].dropna().tolist() if i.strip() != '']
r2_list = [i.strip() for i in df2[f'original_text_isssues_detected'].dropna().tolist() if i.strip() != '']

In [12]:
a = []
for r in r2_list:
  parsed_items = r.lower().split('|')
  for p in parsed_items:
    a.append(p.strip())
print(len(a))
print(len(set(a)))

114
30


In [ ]:
print("reviewer1_issues:", len(r1_list))
print("reviewer2_issues:", len(r2_list))

merged_list = []
merged_list.extend(r1_list)
merged_list.extend(r2_list)

print('merged issues:', len(merged_list))

reviewer1_issues: 10
reviewer2_issues: 65
merged issues: 75


In [ ]:
merged_list

['I pilastri sembrano 5 e non 6',
 "Molte perifrasi | molti nomi d'azione | primo capoverso oscuro",
 'Frase lunga | inutili perifrasi',
 "Spezzare frasi lunghe | evitare astratti e nomi d'azione | ripetere il soggetto",
 'Perifrasi | lunghezze | frasi subordinate implicite',
 'si tratta di paragrafi collegati a un paragrafo precedente che elenca i compiti della Polizia locale',
 "Frasi lunghe | molti nomi d'azione",
 'Non chiaro come questo paragrafo si collega al precedente',
 'Forse manca qualcosa',
 'lessico',
 'frasi lunghe | lessico tecnico',
 'frasi lunghe | passivi | sintassi contorta | latinismi | termini tecnici',
 "linguaggio tecnico dell'economia",
 'termine tecnico privo di equivalente',
 'acronimo',
 'espressioni verbali complesse (decreta la sospensione, dispone la sospensione)',
 'locuzioni verbali',
 'lessico tecnico',
 'latinismo | lessico',
 'lessico ricercato | inciso',
 'locuzioni | lessico burocratico',
 'frasi lunghe | lessico burocratico',
 'acronimo | locuzioni

## Parse list

In [ ]:
items = list()
unique_items = set()
for raw_items in merged_list:
  parsed_items = raw_items.lower().split('|')
  for parsed_item in parsed_items:
    parsed_item = parsed_item.strip()
    if parsed_item != '':
      items.append(parsed_item)
      unique_items.add(parsed_item)

print('total issues:', len(items))
print('total unique issues:', len(unique_items))

total issues: 152
total unique issues: 52


In [ ]:
unique_items

{'acronimi',
 'acronimo',
 'anglismi',
 'cirumlocuzione',
 'cirumlocuzioni',
 'espressioni nominali',
 'espressioni verbali complesse (decreta la sospensione, dispone la sospensione)',
 "evitare astratti e nomi d'azione",
 'forse manca qualcosa',
 'frase lunga',
 'frase scissa',
 'frasi lunghe',
 'frasi subordinate implicite',
 'i pilastri sembrano 5 e non 6',
 'incisi',
 'inciso',
 'inutili perifrasi',
 'latinismi',
 'latinismo',
 'lessico',
 'lessico burocratico',
 'lessico ricercato',
 'lessico tecnico',
 'lessico tecnico del diritto',
 "linguaggio tecnico dell'economia",
 'locuzione',
 'locuzioni',
 'locuzioni burocratiche',
 'locuzioni verbali',
 'lunghezze',
 'molte perifrasi',
 "molti nomi d'azione",
 'non chiaro come questo paragrafo si collega al precedente',
 'ordine marcato',
 'ordini dei costituenti non prototipici',
 'ordini marcati',
 'passivi',
 'passivo',
 'perifrasi',
 'periodo complesso',
 'primo capoverso oscuro',
 'ripetere il soggetto',
 'si tratta di paragrafi col

## Clustering

In [ ]:
clustered_items = do_clustering(list(unique_items))
print('n_clusters:', len(clustered_items))

n_clusters: 16


## Gen title and keywords

In [ ]:
taxonomy = []
for i, items in clustered_items.items():
  taxonomy_item = {
      'id': int(i + 1),
      'items': items,
      'title': gen_title_from_docs_list(items),
      'keywords': do_topic_extractions(items, 2),
  }
  taxonomy.append(taxonomy_item)

## Visualize taxonomy

In [ ]:
for taxonomy_item in taxonomy:
  print(taxonomy_item)

{'id': 1, 'items': ['perifrasi', 'termini desueti', 'locuzione', 'molte perifrasi', 'cirumlocuzioni', 'inutili perifrasi', 'sintassi contorta', 'locuzioni', 'cirumlocuzione', 'lessico', 'lessico ricercato'], 'title': 'giri di parole antichi.', 'keywords': ['perifrasi', 'lessico']}
{'id': 3, 'items': ['termini tecnici', 'termine tecnico', 'lessico tecnico del diritto', 'lessico tecnico', 'termine tecnico privo di equivalente', "linguaggio tecnico dell'economia"], 'title': 'linguaggio tecnico e specifico', 'keywords': ['tecnico', 'lessico']}
{'id': 4, 'items': ['latinismo', 'latinismi', 'anglismi'], 'title': 'espressioni linguistiche di derivazione latina e inglese.', 'keywords': ['anglismi', 'latinismi']}
{'id': 5, 'items': ['espressioni nominali', 'subordinata implicita', 'passivi', 'frasi subordinate implicite', 'stile nominale', 'passivo'], 'title': 'forme passive ed implicite', 'keywords': ['passivi', 'passivo']}
{'id': 14, 'items': ['ordini marcati', 'ordini dei costituenti non pro

## Export

In [ ]:
with open(f'./rules_and_issues/issues.json', 'w', encoding='utf-8') as f:
  json.dump(taxonomy, f, ensure_ascii=False, indent=4)

# Rules

## Merge lists

In [ ]:
r1_list = [i.strip() for i in df[f'simplification_rules_applied'].dropna().tolist() if i.strip() != '']
r2_list = [i.strip() for i in df[f'simplification_rules_applied'].dropna().tolist() if i.strip() != '']

In [ ]:
print("reviewer1_rules:", len(r1_list))
print("reviewer2_rules:", len(r2_list))

merged_list = []
merged_list.extend(r1_list)
merged_list.extend(r2_list)

print('merged rules:', len(merged_list))

reviewer1_rules: 140
reviewer2_rules: 62
merged rules: 202


In [ ]:
merged_list

['Un verbo semplice in luogo di una perifrasi | parole concrete e comuni | indicare il riferimento legge tra parentesi per farlo meglio evidenziare',
 'Portare la citazione di legge tra parentesi e provare a sintetizzare il contenuto prima della parentesi in modo semplice e discorsivo | Usare espressioni comuni come legge chiamata X, riferirsi alla sigla come gruppo di lettere',
 "Riordino di parti | messa in parentesi dei riferimenti legislativi | eliminazione perifrasi verbali a vantaggio di forme verbali semplici e dell'indicativo | Eliminare espressioni di tipo giuridico (ai sensi di ...)",
 "Parole comuni | nomi d'azione sostituiti da verbi | Perifrasi verbali e concetti astratti sostituiti da verbi semplici e concreti",
 'Ente riscritto come Provincia per non confondere | verbo negativo non può prescindere sostituito da forma attiva deve derivare | Spezzare frase lunga | mettere tra parentesi il riferimento alla legge | inserire le maiuscole per identificare le stesse sigle D.Lgs

## Parse list

In [ ]:
items = list()
unique_items = set()
for raw_items in merged_list:
  parsed_items = raw_items.lower().split('|')
  for parsed_item in parsed_items:
    parsed_item = parsed_item.strip()
    if parsed_item != '':
      items.append(parsed_item)
      unique_items.add(parsed_item)

print('total rules:', len(items))
print('total unique rules:', len(unique_items))

total rules: 471
total unique rules: 336


In [ ]:
unique_items

{'abbreviare',
 'abbreviare eliminando perifrasi',
 'abbreviare giri di parole',
 'abbreviare perifrasi',
 'abbreviare perifrasi lunghe',
 'abbreviare testo',
 "abbreviata e spezzata dall'introduzione di due punti",
 'abbreviato molto il testo, eliminando perifrasi ma soprattutto rimettendo in ordine la frase con soggetto a inizio frase, verbo colloquiale, e poi i complementi',
 'abbreviazione',
 'aderisce sostituito con fa parte',
 'affisso sostituito col comune incollato',
 'agevolazione di prezzo diventa prezzi più bassi',
 'aggettivi invece dei nomi astratti di qualità',
 "aggiungere l'ausiliare prima del participio passato per rendere esplicito il modo verbale",
 'aggiungere una ridondanza utile a chiarire',
 'aggiungere verbi esplicativi come scrivere riferito alla relazione',
 'aggiunta del verbo',
 "aggiunta di 'a' dopo telefonando",
 'aggiunta di qualche precisazione sulla lista (presso, su)',
 'aggiunta finale per il riferimento alle conseguenze di legge',
 "anche la seconda 

## Clustering

In [ ]:
clustered_items = do_clustering(list(unique_items))
print('n_clusters:', len(clustered_items))

n_clusters: 42


## Gen title and keywords

In [ ]:
taxonomy = []
for i, items in clustered_items.items():
  taxonomy_item = {
      'id': int(i + 1),
      'items': items,
      'title': gen_title_from_docs_list(items),
      'keywords': do_topic_extractions(items, 2),
  }
  taxonomy.append(taxonomy_item)

## Visualize taxonomy

In [ ]:
for taxonomy_item in taxonomy:
  print(taxonomy_item)

{'id': 9, 'items': ['mettere sempre a inizio un soggetto esplicito e agentivo', 'iniziare frase col soggetto', 'mettere sempre un soggetto esplicito che sia possibilmente uno degli attori in gioco', 'indicare sempre il soggetto esplicitandolo'], 'title': "inserire soggetto all'inizio frase.", 'keywords': ['soggetto', 'sempre']}
{'id': 55, 'items': ['verbo colloquiale rispetto a formale (supportare)', "rendere colloquiale l'espressione", 'sostituzione di parole burocratiche o astratte con espressioni comuni - colloquiali', 'uso di linguaggio comune', 'eliminazione di parole e espressioni molto più formali a vantaggio di parole concrete', 'sostituire parole formali con parole colloquiali', 'spiegazione di sinonimia tra reclamo parola colta e lamentela parola colloquiale', 'spiegare i concetti in modo colloquiale', 'termini colloquiali invece di più formali', 'tentativo di sostituzione con termini non tecnici', 'sostituire termine raro o formale con termine colloquiale', 'espressioni più 

## Export

In [ ]:
with open(f'./rules_and_issues/rules.json', 'w', encoding='utf-8') as f:
  json.dump(taxonomy, f, ensure_ascii=False, indent=4)